## Scraping companies data from investing.com

### Install libraries

In [16]:
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install requests pandas
!{sys.executable} -m pip install selenium playwright


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Users/concept/myenv/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Users/concept/myenv/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Users/concept/myenv/bin/python -m pip install --upgrade pip


## Import libraries

In [18]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
from selenium import webdriver

### Links to parse:

In [138]:
# https://www.investing.com/equities/**trinity-mirror**
# https://www.investing.com/equities/trinity-mirror**-company-profile**
# https://www.investing.com/equities/trinity-mirror**-ownership**

### Get contents of a company info page, and read it into Beautiful Soup

In [132]:
# url = "https://www.investing.com/equities/trinity-mirror"
url = "https://www.investing.com/equities/volkswagen-ag"
# url = "https://www.investing.com/equities/intel-corp"
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# BS only version
# print(soup.prettify())
# data  = requests.get(url).text 
# soup = BeautifulSoup(data,"html.parser")  # create a soup object using the variable 'data'

### Get company name, logo, ticker, headquarters(country), currency, description

In [137]:
logo = soup.find('img', class_='float-left mr-3 mt-1.5 h-5 w-5')
if logo is not None:
    logo = logo.get('src')
print(logo)

company = soup.find('h1', class_='mb-2.5 inline text-left text-xl/7 font-bold text-[#232526] md:mb-2 md:text-3xl/8 rtl:ml-1 rtl:soft-ltr').get_text(strip=True)
print(company)

flag_div = soup.find('div', class_='relative flex cursor-pointer items-center gap-3 border-b-2 border-transparent hover:border-inv-blue-500 hover:text-link')
flag_span = flag_div.find('span', attrs={'class': lambda x: x and x.startswith('flag_flag__gUPtc flag_flag--')})
country_code = flag_span['data-test'].replace('flag-', '')
print(country_code) 

currency = soup.find('div', attrs={'data-test': 'currency-in-label'}).find('span').get_text(strip=True)
print(currency)


None
Volkswagen AG (VOWG)
DE
EUR


### Read specific company data

In [180]:
divs = soup.find_all('div', class_='flex flex-wrap items-center justify-between border-t border-t-[#e6e9eb] pt-2.5 text-xs leading-4 sm:pb-2.5 pb-2.5')

# Step 1: Find span
# prev_close_span = soup.find('span', string='Market Cap')
# prev_close_span = soup.find('span', string='Revenue')
# prev_close_span = soup.find('span', string='Dividend (Yield)') # (%)
# prev_close_span = soup.find('span', string='EPS')
# prev_close_span = soup.find('span', string='P/E Ratio')
prev_close_span = soup.find('span', string='Gross Profit Margin') # %

# prev_close_span = soup.find('span', string='Revenue')
# prev_close_span = soup.find('span', string='Revenue')
# prev_close_span = soup.find('span', string='Revenue')
# prev_close_span = soup.find('span', string='Revenue')
print(prev_close_span)

# Step 2: Get parent container div
container_div = prev_close_span.find_parent('div', class_='flex flex-wrap items-center justify-between border-t border-t-[#e6e9eb] pt-2.5 text-xs leading-4 sm:pb-2.5 pb-2.5')
# print(container_div.prettify())

# Step 3: Find dd element
dd = container_div.find('dd')
# Find all spans with class key-info_dd-numeric__ZQFIs
numeric_containers = dd.find_all('span', class_='key-info_dd-numeric__ZQFIs')

values = []
for container in numeric_containers:
    # Get text from this container only (first numeric child)
    for child_span in container.find_all('span'):
        text = child_span.get_text(strip=True)
        if text:
            try:
                float(text)
                values.append(text)
                break  # Only first numeric per container
            except ValueError:
                pass

print(f"Values: {values}")  # ['value1', 'value2']

# Reusable function
def find_company_data(data):
    '''Extract numeric value of the spcified statistic'''
    data_span = soup.find('span', string=data)
    # print(data_span)
    
    container_div = data_span.find_parent('div', class_='flex flex-wrap items-center justify-between border-t border-t-[#e6e9eb] pt-2.5 text-xs leading-4 sm:pb-2.5 pb-2.5')
    
    dd = container_div.find('dd')
    numeric_containers = dd.find_all('span', class_='key-info_dd-numeric__ZQFIs')
    
    values = []
    for container in numeric_containers:
        # Get text from this container only (first numeric child)
        for child_span in container.find_all('span'):
            text = child_span.get_text(strip=True)
            if text:
                try:
                    float(text)
                    values.append(float(text))
                    break  # Only first numeric per container
                except ValueError:
                    pass
    # return (f"Values: {values}")
    if len(values) > 1:
        return values
    else:
        return values[0]

print(find_company_data('Dividend (Yield)'))
print(find_company_data('EPS'))


<span>Gross Profit Margin</span>
Values: ['15.4']
[6.3, 6.03]
13.4352


### List of available companies - will be generated by AI

In [174]:
companies_list = ['trinity-mirror', 'volkswagen-ag', 'intel-corp', 'and-so-on...']

### Base constants

In [179]:
base_url = 'https://www.investing.com/equities/'
base_url_logo = 'ttps://cdn.investing.com/entities-logos/'

# For company description:
company_profile =  base_url+companies_list[0]+'-company-profile'

# For company ownership:
company_ownership =  base_url+companies_list[0]+'-ownership'

### Data model for companies, & company information

In [ ]:
companies = [
    {
        'name': 'Reach PLC (RCH)',
        'country': 'GB',
        'logo': '876853.svg', # or None
        'currency': 'GBP',
        'url_investing': 'trinity-mirror',
        'Market Cap': 13.4352,
        'Revenue': 13.4352,
        'Dividend (Yield)': [6.30, 6.03],
        'EPS': 13.4352,
        'P/E Ratio': 13.4352,
        'Gross Profit Margin': 13.4352
    },
    {
        'name': 'Volkswagen AG (VOWG)',
        'country': 'DE',
        'logo': 'None',
        'currency': 'EUR',
        'url_investing': 'volkswagen-ag',
        'Market Cap': 13.4352,
        'Revenue': 13.4352,
        'Dividend (Yield)': [6.30, 6.03],
        'EPS': 13.4352,
        'P/E Ratio': 13.4352,
        'Gross Profit Margin': 13.4352
    },
    {
        'name': 'Intel Corporation (INTC)',
        'country': 'US',
        'logo': '21127.svg',
        'currency': 'USD',
        'url_investing': 'intel-corp',
        'Market Cap': 13.4352,
        'Revenue': 13.4352,
        'Dividend (Yield)': [6.30, 6.03],
        'EPS': 13.4352,
        'P/E Ratio': 13.4352,
        'Gross Profit Margin': 13.4352
        }
]